# Transaction Configuration

Shows how transaction configurations are set up and change output transactions and holdings

Attributes
----------
transaction configuration
transactions
output transactions
holdings

This notebook demonstrates how a [transaction configuration](https://support.lusid.com/knowledgebase/article/KA-01872) can be set up in LUSID, and how it can be utilised.

In the example below, we will

<ol> (1) Create a parent portfolio </ol> 
<ol> (2) Create two equity stocks - VOD.L and AMZN</ol> 
<ol> (3) Create an simple instrument with base currency of USD, representing cash reserve in USD</ol> 
<ol> (4) Insert a new transaction to illustrate 100000.00 GBP is added to portfolio</ol> 
<ol> (5) Insert a new transaction to illustrate 100000.00 USD is added to portfolio as cash reserve</ol> 
<ol> (6) Insert a new transaction to buy 250 stocks of VOD.L at price of 1.00 GBP</ol> 
<ol> (7) Insert a new transaction to buy 300 stocks of AMZN at price of 95.00 USD</ol> 

Throughout the notebooke, we will:
<ul> (1) Explore the default transaction configuration available in LUSID, and different elements which form the transaction configuration</ul>
<ol> (2) Get output transactions and holdings of portfolio, and examine how default transaction configuration impact retreived holdings</ol>
<ol> (3) Set up new property definitions to be used in transaction configuration</ol>
<ol> (4) Set up new side definitions, transaction types, and movement types in transaction configuration</ol>
<ol> (5) Apply new movement types to transactions, and examine how the resultant holdings are changed based on new transaction configuration setup</ol> 

## Setup LUSID

In [1]:
# Import general purpose packages
import os
import json
from datetime import datetime
from datetime import timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.exceptions import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

Load a mapping file for DataFrame headers for handling responses from 
+ `build transaction`
+ `get holdings`
+ `list_instruments`

In [2]:
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

with open(r"config/list_instruments_mapping.json") as mappings_file:
    list_instruments_json_mapping = json.load(mappings_file)
    
with open(r"config/transaction_configuration_data_mapping.json") as mappings_file:
    transaction_configuration_data_json_mapping = json.load(mappings_file)

In [3]:
# Create new scope, portfolio code, and dates to be used throughout the notebook

scope="notebook-portfolios"
portfolio_code="EQUITY" + "-" + create_scope_id()
transaction_date=datetime(year=2020, month=10, day=1, tzinfo=pytz.UTC)
settlement_date=datetime(year=2020, month=10, day=5, tzinfo=pytz.UTC)

In [4]:
# Define [transaction portfolio] [instrument] [transaction configuration] API

portfolios_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

### Create portfolio and instruments

In [5]:
# Create a transaction portfolio, with base currency GBP

create_transaction_portfolio_response=transaction_portfolios_api.create_portfolio(
    scope=scope,
    create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
        code=portfolio_code,
        display_name=portfolio_code,
        base_currency='GBP',
        created='2000-01-01T00:00:00+00:00'
    )
)

In [6]:
# Create a VOD.L and AMZN equities as instruments

upsert_instruments_response = instruments_api.upsert_instruments(
    scope='default',
    request_body=
    {
        'create-VOD.L':
        models.InstrumentDefinition(
            name='Vodafone PLC',
            identifiers={
                'Sedol': models.InstrumentIdValue('BH4HKS3'),
                'Isin': models.InstrumentIdValue('GB00BH4HKS39'),
                'ClientInternal': models.InstrumentIdValue('cid_VOD.L')
            },
            definition=models.Equity(
                instrument_type="Equity",
                dom_ccy="GBP",
            )
        ),
        'create-AMZN':
        models.InstrumentDefinition(
            name='Amazon',
            identifiers={
                'Figi': models.InstrumentIdValue('BBG000BVPV84'),
                'ClientInternal': models.InstrumentIdValue('cid_AMZN')
            },
            definition=models.Equity(
                instrument_type="Equity",
                dom_ccy="USD",
            )
        )
    }
)

### Load transactions into portfolio

In [7]:
#First, put in 150,000.00 GBP to the portfolio

upsert_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0001',
            type='FundsIn',
            instrument_identifiers={
                'Instrument/default/Currency':'GBP'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=150000,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=150000,
                currency='GBP'
            )
        )
    ]
)

In [8]:
# Next, buy 250 VOD.L at price of 1.00 GBP for each stock

upsert_vod_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0002',
            type='Buy',
            instrument_identifiers=
            {
                'Instrument/default/Isin': 'GB00BH4HKS39'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=250,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=250,
                currency='GBP'
            )
        )
    ]
)

### Initial Examination of default Transaction Configurations and Resultant Holdings

Let's have a quick look at `movements` of associated with two `transaction types` which we used above - "Buy" and "FundsIn". 

#### Transaction Type `Buy`

"Buy" type indicates stock side (Side1) "moves up" and currency (Side2) "moves down" when buying a stock.


In [9]:
get_buy_transaction_configuration_response = transaction_configuration_api.get_transaction_type(
    source='default',
    type='Buy'
)

lusid_response_to_data_frame(get_buy_transaction_configuration_response.movements)

,movement_types,side,direction,properties,mappings,movement_options
0,StockMovement,Side1,1,{},[],[]
1,CashCommitment,Side2,-1,{},[],[]


#### Transaction Type `FundsIn`

'FundsIn' states only 'funds' go 'into' the portfolio

In [10]:
get_funds_in_transaction_configuration_response = transaction_configuration_api.get_transaction_type(
    source='default',
    type='FundsIn'
)

lusid_response_to_data_frame(get_funds_in_transaction_configuration_response.movements)

,movement_types,side,direction,properties,mappings,movement_options
0,CashReceivable,Side1,1,{},[],[]


#### Holdings after buying 250 VOD.L @ £1

When getting holdings for portfolio, you should see two holdings: 

1) 250 unit of VOD.L @ £1, with total cost of £250

2) £149750 in portfolio, because 
  - £150000 of GBP as "funds" got "in"
  - £250 of GBP as gone out to buy those 250 VOD.L stocks

In [11]:
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(get_holdings_response)

,instrument_scope,instrument_uid,sub_holding_keys,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,properties.Instrument/default/Name.effective_from,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003D6P,{},Instrument/default/Name,Vodafone PLC,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,CCY_GBP,{},Instrument/default/Name,GBP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance


## Upsert Transaction as USD Cash Reserve

### Create an instrument to represent as USD Cash Reserve

First, create an instrument which represents it is a USD Cash Reserve

In [12]:
upsert_usd_cash_reserve_response = instruments_api.upsert_instruments(
    scope='default',
    request_body=
    {
        'create-cash-reserve-usd':
        models.InstrumentDefinition(
            name='Cash Reserve (USD)',
            identifiers={
                'ClientInternal': models.InstrumentIdValue('cid_cash_reserve_usd')
            },
            definition=models.SimpleInstrument(
                asset_class='Money',
                dom_ccy='USD',
                simple_instrument_type='None',
                instrument_type='SimpleInstrument'
            )
        )
    }
)

# in addition, we will keep record of LusidInstrumentId for that USD Cash Reserve for later use.

usd_cash_reserve_luid = upsert_usd_cash_reserve_response.values['create-cash-reserve-usd'].identifiers['LusidInstrumentId']

### Insert transaction with USD Cash Reserve to portfolio

Next, we will first to "buy" $100,000 USD as cash reserve.

In [13]:
upsert_usd_cash_reserve_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0003',
            type='Buy',
            instrument_identifiers={
                'Instrument/default/ClientInternal':'cid_cash_reserve_usd'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=100000,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=100000,
                currency='USD'
            )
        )
    ]
)

### Examine holdings after inserting transaction with USD Cash Reserve

Here, you can see the cash reserve is treated as "buying an instrument", and 100,000 USD is being deducted. As a result, there is an additional "holding" that states 100,000 USD is deducted. 

However, this is not what the transaction is implied, because we are adding 100,000 USD as cash reserve.

In [14]:
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(get_holdings_response)

,instrument_scope,instrument_uid,sub_holding_keys,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,properties.Instrument/default/Name.effective_from,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003D6P,{},Instrument/default/Name,Vodafone PLC,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,LUID_00003D6Q,{},Instrument/default/Name,Cash Reserve (USD),0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,100000.0,100000.0,100000.0,USD,0.0,GBP,USD,Position
2,default,CCY_GBP,{},Instrument/default/Name,GBP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance
3,default,CCY_USD,{},Instrument/default/Name,USD,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,-100000.0,-100000.0,-100000.0,USD,0.0,GBP,USD,Balance


To fix, set up an alternate `movement type` "CashReserveIn" to indicate this is as "adding $100000 USD cash as cash reserve".

## Create "CashReserveIn" transaction configuration type and movement

### Create a new transaction type "CashReserveIn".

First, create a new `trasaction type` called "CashReserveIn".

`Movements` of this `transaction type` is same as that in "FundsIn" - a cash receivable with positive movement.

In [15]:
usd_cash_reserve_in_txn_type_response = transaction_configuration_api.set_transaction_type(
    source='default',
    type='CashReserveIn',
    transaction_type_request=models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type='CashReserveIn',
                description='Adding funds as cash reserve',
                transaction_class='FundsIn',
                transaction_roles="Longer"
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types='CashReceivable',
                side='Side1',
                direction=1
            ),
        ]
    )
)

### Update transaction with Transaction Type "CashReserveIn"

Upsert the same transaction again, but only change transaction type as "CashReserveIn".

In [16]:
upsert_usd_cash_reserve_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0003',
            type='CashReserveIn',
            instrument_identifiers={
                'Instrument/default/ClientInternal':'cid_cash_reserve_usd'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=100000,
            transaction_price=models.TransactionPrice(
                price=1,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=100000,
                currency='USD'
            )
        )
    ]
)

### Examine holdings after "CashReserveIn" transaction type is set up and used

If we get holdings now again, you can see the USD cash reserve is added as "new funds is added".

In [17]:
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(get_holdings_response)

,instrument_scope,instrument_uid,sub_holding_keys,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,properties.Instrument/default/Name.effective_from,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003D6P,{},Instrument/default/Name,Vodafone PLC,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,CCY_GBP,{},Instrument/default/Name,GBP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance
2,default,LUID_00003D6Q,{},Instrument/default/Name,Cash Reserve (USD),0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,100000.0,100000.0,100000.0,USD,0.0,GBP,USD,Position


## Setting Transaction Configuration and Movements dependent on property values

First, buy 300 Amazon stocks @ $95 USD.

In [18]:
upsert_amzn_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0004',
            type='Buy',
            instrument_identifiers={
                'Instrument/default/ClientInternal':'cid_AMZN'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=300,
            transaction_price=models.TransactionPrice(
                price=95,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=28500,
                currency='USD'
            )
        )
    ]
)

Now, get holdings for the portfolio.

You should see there are now 5 holdings; Note the holding with instrument CCY_USD with -28,500 USD.

This is because we use `transaction type` "Buy" to buy those 300 Amazon stocks; our default transaction configuration setup would indicate to take away USD cash (CCY_USD) when doing so. 

In [19]:
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(get_holdings_response)

,instrument_scope,instrument_uid,sub_holding_keys,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,properties.Instrument/default/Name.effective_from,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003D6P,{},Instrument/default/Name,Vodafone PLC,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,LUID_00003D6R,{},Instrument/default/Name,Amazon,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,300.0,300.0,28500.0,USD,0.0,GBP,USD,Position
2,default,CCY_GBP,{},Instrument/default/Name,GBP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance
3,default,LUID_00003D6Q,{},Instrument/default/Name,Cash Reserve (USD),0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,100000.0,100000.0,100000.0,USD,0.0,GBP,USD,Position
4,default,CCY_USD,{},Instrument/default/Name,USD,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,-28500.0,-28500.0,-28500.0,USD,0.0,GBP,USD,Balance


However, there is already some USD Cash Reserve available in Portfolio; and we would like to use that cash reserve to buy Amazon stocks. 

We can use additional setup to achieve this. 

### Set Up Transaction Configuration to Deduct from Cash Reserve

First, create a new property definition for later use. 

This property will be stamped on the transaction if the funds needs to be deducted from a specific cash reserve.

In [20]:
try: 
    create_deduct_from_cash_reserve_luid_property_definition_response = property_definitions_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain='Transaction',
            scope='TradeWithCashReserve',
            code='DeductFromLuid',
            display_name='Deduct from Cash Reserve LUID',
            data_type_id=models.ResourceId('system','string'),
            life_time='Perpetual',
            constraint_style='Property',
            property_description='Cash Reserve LUID to deduct funds from when trading',
        )
    )
except: 
    pass

Next, create a new [`side definition`](https://support.lusid.com/knowledgebase/article/KA-01875/) to indicate deducting from cash reserve needs to take place.

This side definition references security to use 'Transaction/TradeWithCashReserve/DeductFromLuid' property to state which cash reserve to deduct from.

In [21]:
buy_from_cash_reserve_side_def_response = transaction_configuration_api.set_side_definition(
    side='DeductFromCashReserve',
    side_definition_request=models.SideDefinitionRequest(
        security='Transaction/TradeWithCashReserve/DeductFromLuid',
        currency='Txn:TradeCurrency',
        rate='Txn:TradeToPortfolioRate',
        units='Txn:TotalConsideration',
        amount='Txn:TotalConsideration'
    )
)

Lastly, create a new transaction type "notebook-portfolios-BuyFromCashReserve" in "default" source. 

When this transaction type is used, it states to deduct from cash reserve stated in 'Transaction/TradeWithCashReserve/DeductFromLuid' property in a transaction.

In [22]:
buy_from_cash_reserve_in_txn_type_response = transaction_configuration_api.set_transaction_type(
    source='default',
    type='notebook-portfolios-BuyFromCashReserve',
    transaction_type_request=models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type='notebook-portfolios-BuyFromCashReserve',
                description='Buy from Cash Reserve',
                transaction_class='Basic',
                transaction_roles='LongLonger'
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types='StockMovement',
                side='Side1',
                direction=1
            ),
            models.TransactionTypeMovement(
                movement_types='CashCommitment',
                side='DeductFromCashReserve',
                direction=-1
            )
        ]
    )
)

### Upsert 'Buy Amazon' Transaction with type "BuyFromCashReserve"

Now, we upsert the same transaction for buying Amazon stocks with two changes: 

1) Transaction Type is now 'notebook-portfolios-BuyFromCashReserve', which was set up previously.

2) This transaction has an additional property key 'Transaction/TradeWithCashReserve/DeductFromLuid'. The value of this property is the LusidInstrumentId (LUID) of the same USD Cash Reserve we created earlier.

In [23]:
upsert_amzn_transaction_response=transaction_portfolios_api.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_request=[
        models.TransactionRequest(
            transaction_id='trd-0004',
            type='notebook-portfolios-BuyFromCashReserve',
            instrument_identifiers={
                'Instrument/default/ClientInternal':'cid_AMZN'
            },
            transaction_date=transaction_date.isoformat(),
            settlement_date=settlement_date.isoformat(),
            units=300,
            transaction_price=models.TransactionPrice(
                price=95,
                type='Price'
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=28500,
                currency='USD'
            ),
            properties={
                'Transaction/TradeWithCashReserve/DeductFromLuid':
                models.PerpetualProperty(
                    key='Transaction/TradeWithCashReserve/DeductFromLuid',
                    value=models.PropertyValue(
                        label_value=usd_cash_reserve_luid
                    )
                )
            }
        )
    ]
)

### Examine holdings after update transaction with "BuyFromCashReserve" transaction type

Now, get holdings again for the portfolio. 

After the 'Buying Amazon stocks' transaction is updated to use "BuyFromCashReserve" transaction type, you should see there are 4 holdings instead of 5. The holding with instrument "CCY_USD" no longer exists.

Note the holding with instrument name "Cash Reserve (USD)". The amount of that holding is now gone down to 71,500 USD. This is because "BuyFromCashReserve" transaction type has indicated to take the USD Cash Reserve to buy those 300 Amazon stock, which costs 28,500 USD in total. 

This leaves with (100,000 - 28,500 = ) 71,500 USD Cash Reserve left.

In [24]:
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

lusid_response_to_data_frame(get_holdings_response)

,instrument_scope,instrument_uid,sub_holding_keys,properties.Instrument/default/Name.key,properties.Instrument/default/Name.value.label_value,properties.Instrument/default/Name.effective_from,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,holding_type_name
0,default,LUID_00003D6P,{},Instrument/default/Name,Vodafone PLC,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,250.0,250.0,250.0,GBP,250.0,GBP,GBP,Position
1,default,LUID_00003D6R,{},Instrument/default/Name,Amazon,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,300.0,300.0,28500.0,USD,0.0,GBP,USD,Position
2,default,CCY_GBP,{},Instrument/default/Name,GBP,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,B,149750.0,149750.0,149750.0,GBP,149750.0,GBP,GBP,Balance
3,default,LUID_00003D6Q,{},Instrument/default/Name,Cash Reserve (USD),0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioId,EQUITY-3b9a-3b23-8a8d-b6,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,notebook-portfolios,0001-01-01 00:00:00+00:00,P,71500.0,71500.0,71500.0,USD,0.0,GBP,USD,Position
